In [1]:
import torch
from torchtext.datasets import AG_NEWS

train_iter = iter(AG_NEWS(split="train"))

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")
train_iter = AG_NEWS(split="train")


def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

vocab(['here', 'is', 'an', 'example'])


text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


train_iter = AG_NEWS(split="train")
dataloader = DataLoader(
    train_iter, batch_size=8, shuffle=False, collate_fn=collate_batch
)

In [2]:
train_iter

ShardingFilterIterDataPipe

In [14]:
for i in train_iter: 
    print(i)
    break

(3, "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")


c:\Users\kevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\datapipes\iter\combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "


In [3]:
for i in dataloader:
    print(i)
    break

(tensor([2, 2, 2, 2, 2, 2, 2, 2]), tensor([  431,   425,     1,  1605, 14838,   113,    66,     2,   848,    13,
           27,    14,    27,    15, 50725,     3,   431,   374,    16,     9,
        67507,     6, 52258,     3,    42,  4009,   783,   325,     1, 15874,
         1072,   854,  1310,  4250,    13,    27,    14,    27,    15,   929,
          797,   320, 15874,    98,     3, 27657,    28,     5,  4459,    11,
          564, 52790,     8, 80617,  2125,     7,     2,   525,   241,     3,
           28,  3890, 82814,  6574,    10,   206,   359,     6,     2,   126,
            1,    58,     8,   347,  4582,   151,    16,   738,    13,    27,
           14,    27,    15,  2384,   452,    92,  2059, 27360,     2,   347,
            8,     2,   738,    11,   271,    42,   240, 51953,    38,     2,
          294,   126,   112,    85,   220,     2,  7856,     6, 40066, 15380,
            1,    70,  7376,    58,  1810,    29,   905,   537,  2846,    13,
           27,    14,    27, 

c:\Users\kevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\datapipes\iter\combining.py:333: UserWarning: Some child DataPipes are not exhausted when __iter__ is called. We are resetting the buffer and each child DataPipe will read from the start again.
  warnings.warn("Some child DataPipes are not exhausted when __iter__ is called. We are resetting "
